## <p style="font-family:JetBrains Mono; font-weight:normal; letter-spacing:2px; color:#4E4FEB; font-size:140%; text-align:left;padding: 0px; border-bottom: 3px solid #4E4FEB">Libraries</p>

In [37]:
import warnings
warnings.filterwarnings("ignore")

from selenium import webdriver
import chromedriver_autoinstaller
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ActionChains

from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
import json
from tqdm import tqdm, tqdm_notebook
import time

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.auto import tqdm
from pandas.io.formats.style import Styler

pd.options.display.max_columns = 999
pd.options.display.max_colwidth = 999

tqdm.pandas()

rc = {
    "axes.facecolor": "#F8F8F8", 
    "figure.facecolor": "#F8F8F8", 
    "axes.edgecolor": "#000000",  
    "grid.color": "#EBEBE7" + "30",
    "font.family": "serif",
    "axes.labelcolor": "#000000",
    "xtick.color": "#000000", 
    "ytick.color": "#000000",
    "grid.alpha": 0.4 
}

sns.set(rc=rc) 
palette = ['#ff7f50', '#ffd700', '#ffdab9', '#9fe2bf',
           '#d2b48c', '#008080', '#98ff98', '#000080']


from colorama import Style, Fore 
blk = Style.BRIGHT + Fore.BLACK
gld = Style.BRIGHT + Fore.YELLOW
grn = Style.BRIGHT + Fore.GREEN
red = Style.BRIGHT + Fore.RED
blu = Style.BRIGHT + Fore.BLUE
res = Style.RESET_ALL

import json

In [38]:
def magnify(is_test : bool = False): 
    base_color = '#b57edc'
    if is_test:
        highlight_target_row = []
    else:
        highlight_target_row = dict(selector = 'tr:last-child',
                            props = [('background-color', f'{base_color}' + '20')]) 
    
    return [dict(selector="th", 
                props=[("font-size", "11pt"),
                    ('background-color', f'{base_color}'),
                    ('color', 'white'),
                    ('font-weight', 'bold'),
                    ('border-bottom', '0.1px solid white'), 
                    ('border-left', '0.1px solid white'), 
                    ('text-align', 'right')]),
        
            dict(selector='th.blank.level0', 
                props=[('font-weight', 'bold'),
                        ('border-left', '1.7px solid white'),
                        ('background-color', 'white')]),

            dict(selector="td", 
                    props=[('padding', "0.5em 1em"), 
                        ('text-align', 'right')]),

            dict(selector="th:hover",
                    props=[("font-size", "14pt")]),

            dict(selector="tr:hover td:hover",
                    props=[('max-width', '250px'),
                        ('font-size', '14pt'),
                        ('color', f'{base_color}'),
                        ('font-weight', 'bold'),
                        ('background-color', 'white'),
                        ('border', f'1px dashed {base_color}')]),
            
            dict(selector="caption", 
                props=[(('caption-side', 'bottom'))])] + highlight_target_row

def stylize_simple(df: pd.DataFrame, caption: str) -> Styler:
    """
        Args:
            df: any dataframe (train/test/origin)

        Returns:
            s: the dataframe wrapped into Styler.
    """
    s = df
    s = s.style.set_table_styles(magnify(True)).set_caption(f"{caption}")
    return s

## <p style="font-family:JetBrains Mono; font-weight:normal; letter-spacing:2px; color:#4E4FEB; font-size:140%; text-align:left;padding: 0px; border-bottom: 3px solid #4E4FEB">Intro</p>

Collecting Tokyo Real Estate Case Data

 <a href = 'http://www.oshimaland.co.jp'> 오시마랜드(Oshimaland)</a>
>This website showcases buildings with unfortunate past incidents such as suicide cases, corpse disposal incidents, and murder cases.


## <p style="font-family:JetBrains Mono; font-weight:normal; letter-spacing:2px; color:#4E4FEB; font-size:140%; text-align:left;padding: 0px; border-bottom: 3px solid #4E4FEB">Data Crawling</p>

In [39]:
chrome_path = chromedriver_autoinstaller.install()
driver = webdriver.Chrome(chrome_path)
url = 'https://www.oshimaland.co.jp/'
driver.get(url)
driver.implicitly_wait(10)

In [40]:
main_district = pd.read_csv('/Users/genie/PycharmProjects/PROJECT/AirbnbWise/Oshimaland_data/jieun/main_district_df2.csv')
display(stylize_simple(main_district.head(4), 'main_district'))

,neighbourhood_cleansed,count
0,Shinjuku Ku,2278
1,Taito Ku,1597
2,Sumida Ku,1290
3,Toshima Ku,1002


In [41]:
regionList = list(main_district['neighbourhood_cleansed'])
print(regionList)

['Shinjuku Ku', 'Taito Ku', 'Sumida Ku', 'Toshima Ku', 'Shibuya Ku', 'Minato Ku', 'Setagaya Ku', 'Ota Ku', 'Nakano Ku', 'Chuo Ku', 'Kita Ku', 'Katsushika Ku', 'Suginami Ku', 'Koto Ku', 'Edogawa Ku', 'Bunkyo Ku', 'Arakawa Ku', 'Itabashi Ku', 'Shinagawa Ku', 'Chiyoda Ku', 'Adachi Ku', 'Meguro Ku', 'Nerima Ku', 'Fuchu Shi', 'Hachioji Shi', 'Hino Shi', 'Kokubunji Shi', 'Machida Shi', 'Ome Shi', 'Chofu Shi', 'Musashino Shi', 'Akiruno Shi', 'Mitaka Shi', 'Koganei Shi', 'Higashimurayama Shi', 'Tama Shi', 'Kunitachi Shi', 'Komae Shi', 'Nishitokyo Shi', 'Tachikawa Shi', 'Kodaira Shi', 'Hamura Shi', 'Musashimurayama Shi', 'Okutama Machi', 'Akishima Shi', 'Fussa Shi']


In [42]:
json_url = 'https://www.oshimaland.co.jp/d/3q5znamu.json'
url = Request(json_url, headers={"User-Agent": "Mozilla/5.0"})
code = urlopen(url)
soup = BeautifulSoup(code, 'html.parser')
json_data = json.loads(soup.text)
json_data

{'key': '3q5znamu',
 'lat': 35.693886004366256,
 'lng': 139.70346762941995,
 'info': '飛び降り自殺',
 'ad': '東京都新宿区歌舞伎町一丁目4',
 'dt': '令和2年7月10日',
 'cr': '令和4年3月7日',
 'images': [],
 'links': []}

In [43]:
令和2年7月10日
東京都新宿区歌舞伎町一丁目4
飛び降り自殺
投稿年月日 令和4年3月7日

SyntaxError: invalid syntax (2694355521.py, line 4)